## 특수문자 EDA (MRC 팀 진행)
- Train 폴더의 train, valid set의 question, context, answer 에 포함된 특수문자
- Answer 에는 포함되지 않지만 Context 에만 있는 특수문자에 대해서...!

In [1]:
import pandas as pd
import re
from datasets import load_from_disk


datasets = load_from_disk("/opt/ml/data/train_dataset")

train_df = pd.DataFrame(datasets["train"])
val_df = pd.DataFrame(datasets["validation"])

# == Answers ====================================================================================

def extract_answer_specials(df:pd.DataFrame):
    answer_specials = []
    for i, row in df.iterrows():
        text = row.answers["text"][0]
        result = re.findall(r"[^ㄱ-ㅎ가-힣A-Za-z\d\s]+", text)
        if len(result) > 0:
            answer_specials.extend(result)
    answer_specials = list(set(answer_specials))
    return answer_specials

train_answer_sepcials = extract_answer_specials(train_df)  # train set의 answers에 포함된 특수문자
val_answer_sepcials = extract_answer_specials(val_df)  # validation set의 answers에 포함된 특수문자


# == Context ====================================================================================
def extract_context_specials(df:pd.DataFrame):
    context_specials = []
    for text in df["context"]:
        # 중국어: 一-龥 / 일본어: ぁ-ゔァ-ヴー々〆〤 / 러시아어(적용 X): \u0400-\u04FF
        result = re.findall(r"[^ㄱ-ㅎ가-힣A-Za-z\d\s一-龥ぁ-ゔァ-ヴー々〆〤]+", text)
        if len(result) > 0:
            context_specials.extend(result)
    context_specials = list(set(context_specials))
    return context_specials

train_context_sepcials = extract_context_specials(train_df)  # train set의 context에 포함된 특수문자
val_context_sepcials = extract_context_specials(val_df)  # validation set의 context에 포함된 특수문자


print("train set의 answers에 포함된 특수문자를 제외한 context에 포함된 특수문자:")
train_specials = sorted([special for special in train_context_sepcials if special not in train_answer_sepcials])
print(train_specials)
print()

print("validation set의 answers에 포함된 특수문자를 제외한 context에 포함된 특수문자:")
val_specials = sorted([special for special in val_context_sepcials if special not in val_answer_sepcials])
print(val_specials)

train set의 answers에 포함된 특수문자를 제외한 context에 포함된 특수문자:
['!!', '!!!', '!!\\', '!!》:', '!"\\', "!'", '!)', '!>', '!\\', '!’', '!’(', '!’\\', '!”', '!”\\', '!」', '""', '"\'', '"(', '"("', '"(-', '")', '").', '")\\', '",', '"-', '".', '"...', '"/>', '">', '"\\', '"|', '"~', '"·"', '"é', '"НІ"', '"”', '"《', '#', '$', '%"', '%)', '%,', '%~', '&', "'(", "'('", "')", "')\\", "',", "'-", "'é", "'œ", "'》", "'ㅚ'", "'ㅟ'", '("', '(""', '(#', "('", '()', '(*', '(+)', '(-)', '(--', '(.', '(...)', '(:', '(<', '(=', '(?)', '([ˈ', '(~', '(É', '(Þ', '(Þó', '(ó', '(ə)', '(Αθηναι)', '(Εὔφορβος)', '(Κένταυροι)', '(Μήδεια)', '(Στύξ,', '(Φάλαγγα)', '(Φαίδρα)', '(εὔδοξος)', '(κατάφρακτοι)', '(κατάφρακτος),', '(φαιδρός)', '(Василия', '(Лев)', '(Народничество)', '(Ю́рий', '(княз)"', '(козаки́|', '(нон)', '(מׇשִׁיחַ|', '(إمام,', '(الكاشغري)', '(المعتزلة\u200e)', '(تقیة', '(حزب', '(حصروم)', '(كربغا,', '(ملك', '(சோழ', '(გონიოს', '(დავით', '(ᛚᚢᚴᛁ;', '(ὁπλίτης)', '(‘', '(“', '(…)', '(←', '(→', '(≪', '(①', '(□□□', '(〈',

In [2]:
print(train_answer_sepcials)
print(val_answer_sepcials)

['(成田のようにならないようにしよう', ')"', '(郵務學堂)', '”', '(釋總)', '(片刀)', ',', '＞', ')》', '(康平)', '(扶餘縣監)', '(有限性)', '「', '(魯陵)', '(祭壇)', "'", 'ä', '·', '〉(', '’(鉱害)', "(海蓮之塔)'", '》', '(受信)', '(北京)', '」', '〉', '‘', '(循資格式)', '〈', '(淵太祚)', '(蔣儼)', '(鄭夔弼)', '(塔身)', '°', '(流)', '>', '(', '(陳理)', '(軍毅)', '(原田甚次)', ')≫', '(公)〉', '(遷)', '(媒槪念)', '(冬牲)', '"', '!"', '\xad', '(狼孟)', '!', '(華陽)', '(呂超)', '(部分割卵)"', '(金學奎)', '/', '・', '’(前意識)', '~', '“', '(大)', ':', '(一心)’', '%', '《', '(華淸池)', '<', '’', ')', '二月河)', '(劉肥)', '』', '(日坂駅)', '(重)', '》(', '.', '＜', '.”', '-', ')”', '(愛琿城,', '(宋公祠)', '『', '≫', '(張勳)', '(南雲忠一)', '...', '‘∧’', '(文藏臺)', '+', '(麻辣)', '≪', '(趙匡)', '(優塡王)', '(興輪寺)', '(汪)']
['.', '＜', '"(', '》', "'(", '”', "(陳田)'", ',', '『', '＞', '“', '(齊西之戰)', ':', '〉', '‘', '《', '〈', '’', '(楊黨)', ')', 'ć)', '(', '(黑色肉)', '(呂正鉉)', "'", '』', '"', '(物的成果)', '·']


In [3]:
def extract_question_specials(df:pd.DataFrame):
    question_specials = []
    for text in df["question"]:
        # 중국어: 一-龥 / 일본어: ぁ-ゔァ-ヴー々〆〤 / 러시아어(적용 X): \u0400-\u04FF
        result = re.findall(r"[^ㄱ-ㅎ가-힣A-Za-z\d\s一-龥ぁ-ゔァ-ヴー々〆〤]+", text)
        if len(result) > 0:
            question_specials.extend(result)
    question_specials = list(set(question_specials))
    return question_specials


train_question_sepcials = extract_question_specials(train_df)
val_question_sepcials = extract_question_specials(val_df)

print(train_question_sepcials, '\n'*2, val_question_sepcials)

['.', '-', '!', '?-', '》', '/', '”', '#', '<<', '(حصروم)', '?', ',', '『', '」', '~', '“', '≫', ':', '%', '〉', '‘', '《', '〈', '<', '>>', ')', '≪', '>', '(', '「', ')>', "'", '』', '"', '·'] 

 ['?', ',', '.', '~', "'", '"', '》', '%', '>', '?"', '《', '<']


- 단순하게는 answer 에는 포함되어 있는 특수문자 제거 가능...!
- 이는 학습이나 추론 단계에서 

## Dense Passage Retriever 의 Tokenizer 결과 확인

In [4]:
from transformers import AutoTokenizer

tokenizer1 = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer2 = AutoTokenizer.from_pretrained("klue/bert-base")

In [5]:
print(tokenizer1,'\n'*2 ,tokenizer2)

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}) 

 PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [6]:
import json

with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
            wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()])) 

In [7]:
## get seq lengths after tokenization
seq_lengths_tokenizer1 = [len(tokenizer1(c)['input_ids']) for c in contexts]
seq_lengths_tokenizer2 = [len(tokenizer2(c)['input_ids']) for c in contexts]

Token indices sequence length is longer than the specified maximum sequence length for this model (1337 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


In [8]:
import numpy as np
def show_dist(seq_len):
    total = len(seq_len)
    over = sum([1 for i in seq_len if i > 512])
    over1000 = sum([1 for i in seq_len if i > 1000])
    over1300 = sum([1 for i in seq_len if i > 1300])
    over1600 = sum([1 for i in seq_len if i > 1600])
    min,max = np.min(seq_len),np.max(seq_len)

    print(f'Percentage over 512 tokens: {100*over/total:.2f}%')
    print(f'Percentage over 1000 tokens: {100*over1000/total:.2f}%')
    print(f'Percentage over 1300 tokens: {100*over1300/total:.2f}%')
    print(f'Percentage over 1600 tokens: {100*over1600/total:.2f}%')
    print(f'Minimum: {min}, Maximum: {max}')


In [9]:
# For wiki.contexts
print(f'***For bert-base-multilingual-cased tokenizer***')
show_dist(seq_lengths_tokenizer1)

print('_'*100)

print(f'***For klue/bert-base tokenizer***')
show_dist(seq_lengths_tokenizer2)

***For bert-base-multilingual-cased tokenizer***
Percentage over 512 tokens: 25.35%
Percentage over 1000 tokens: 5.54%
Percentage over 1300 tokens: 2.75%
Percentage over 1600 tokens: 1.55%
Minimum: 74, Maximum: 29113
____________________________________________________________________________________________________
***For klue/bert-base tokenizer***
Percentage over 512 tokens: 19.89%
Percentage over 1000 tokens: 4.16%
Percentage over 1300 tokens: 2.03%
Percentage over 1600 tokens: 1.13%
Minimum: 56, Maximum: 27541


In [10]:
sorted(seq_lengths_tokenizer1, reverse = True)[:10]

[29113, 25834, 17534, 16920, 16284, 12380, 12116, 10834, 10721, 9567]

In [11]:
from datasets import load_from_disk

dataset = load_from_disk("../data/train_dataset")
df = dataset['train']['context']

print(len(df))

3952


In [12]:
## get seq lengths after tokenization
seq_lengths_tokenizer1_ = [len(tokenizer1(c)['input_ids']) for c in df]
seq_lengths_tokenizer2_ = [len(tokenizer2(c)['input_ids']) for c in df]

In [13]:
# For train contexts

print(f'***For bert-base-multilingual-cased tokenizer***')
show_dist(seq_lengths_tokenizer1_)

print('_'*100)

print(f'***For klue/bert-base tokenizer***')
show_dist(seq_lengths_tokenizer2_)


***For bert-base-multilingual-cased tokenizer***
Percentage over 512 tokens: 49.67%
Percentage over 1000 tokens: 6.48%
Percentage over 1300 tokens: 0.20%
Percentage over 1600 tokens: 0.00%
Minimum: 295, Maximum: 1336
____________________________________________________________________________________________________
***For klue/bert-base tokenizer***
Percentage over 512 tokens: 35.86%
Percentage over 1000 tokens: 2.68%
Percentage over 1300 tokens: 0.00%
Percentage over 1600 tokens: 0.00%
Minimum: 241, Maximum: 1174


### 해결방안...?
1. Cutoff 기준으로 가운데 512?
2. Cutoff 기준으로 random 한 starting point 설정 후 512개 사용..!
3. 중요 문장 혹은 paraphrase 한 passage를 사용하면 안될까...? -> ex> pororo나 mecab morph 활용
3. 아니면 일단 접고 성능 더 좋은 Sparse Retrieval 혹은 Elastic Search 기반의 서칭 방법 구축 돕기

In [14]:
df= dataset['train']['question']

## get seq lengths for questions after tokenization
seq_lengths_tokenizer1_q = [len(tokenizer1(c)['input_ids']) for c in df]
seq_lengths_tokenizer2_q = [len(tokenizer2(c)['input_ids']) for c in df]

# For train questions
print(f'***For bert-base-multilingual-cased tokenizer***')
show_dist(seq_lengths_tokenizer1_q)

print('_'*100)

print(f'***For klue/bert-base tokenizer***')
show_dist(seq_lengths_tokenizer2_q)


***For bert-base-multilingual-cased tokenizer***
Percentage over 512 tokens: 0.00%
Percentage over 1000 tokens: 0.00%
Percentage over 1300 tokens: 0.00%
Percentage over 1600 tokens: 0.00%
Minimum: 8, Maximum: 52
____________________________________________________________________________________________________
***For klue/bert-base tokenizer***
Percentage over 512 tokens: 0.00%
Percentage over 1000 tokens: 0.00%
Percentage over 1300 tokens: 0.00%
Percentage over 1600 tokens: 0.00%
Minimum: 7, Maximum: 45
